# IMPORTS

In [ ]:
import os
import pandas as pd
import time
import random
import getpass
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from tqdm import tqdm

# SETTING DRIVER

```python
def driver():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=s, options=options)
```

In [4]:
from selenium import webdriver

# Configurar el WebDriver para Google Chrome
options = webdriver.ChromeOptions()
options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
wd = webdriver.Chrome(options=options)

# Abrir una página web
url ='https://www.linkedin.com/jobs/search/?currentJobId=3652284026&geoId=107025191&keywords=data%20analyst&location=Barcelona%2C%20Catalonia%2C%20Spain&refresh=true'

wd.get(url)

no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, "h1>span").get_attribute('innerText'))

wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')

The version of chrome cannot be detected. Trying with latest driver version


<selenium.webdriver.remote.webelement.WebElement (session="17eafbd81232a10e0da1597b3ab661f2", element="E0C733E79CCB49FF45F665E48C052578_element_2")>

In [5]:
wd.get(url)

# Browse all the jobs

In [6]:
no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, "h1>span").get_attribute('innerText'))

In [7]:
wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')

<selenium.webdriver.remote.webelement.WebElement (session="17eafbd81232a10e0da1597b3ab661f2", element="9B9C0111B96A1FFB8B7A99413CD0530E_element_4")>

In [8]:
no_of_jobs

2000

In [9]:
def enconding_url (job_title, location, base_url):
    
    base_url = "https://www.linkedin.com/jobs/search"
    
    params = {
        "keywords": job_title,
        "location": location,
    }

    encoded_params = "&" + "&".join([f"{quote(k)}={quote(v)}" for k, v in params.items()])
    url_with_params = f"{base_url}{encoded_params}"

    return url_with_params

In [10]:
from selenium import webdriver
import os
import pandas as pd
import time
import random
import getpass
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urlencode
from urllib.parse import quote

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException


def scraping_cards_on_the_left(job_title="data", location="Barcelona, Catalonia, Spain"):
    options = webdriver.ChromeOptions()
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    wd = webdriver.Chrome(options=options)
    
    url = 'https://www.linkedin.com/jobs/search/?currentJobId=3652284026&geoId=107025191&keywords=data%20analyst&location=Barcelona%2C%20Catalonia%2C%20Spain&refresh=true'
    
    wd.get(url)
    no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, "h1>span").get_attribute('innerText'))
    print("no_of_jobs: ", no_of_jobs)
    wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')
    
    jobsitos = []
    els_jobs = []
    i = 1

    previous_page_content = ""  # Variable para almacenar el contenido de la página anterior

    while i <= int(no_of_jobs/25) + 1:
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        i = i + 1
        try:
            job_lists = wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')
            listita = job_lists.get_attribute('outerHTML')
            soup = BeautifulSoup(listita, "html.parser")
            soup_2 = soup.find_all("ul", {"class": "jobs-search__results-list"})

            # 1. EXTRAER JOB A JOB
            for pagina in soup_2:
                jobsitos.append(pagina)

            # 2. EXTRAER LA INFO DE CADA JOB
            for job in jobsitos:
                job_list = job.find_all("li")

                for the_one_job in job_list:
                    url = the_one_job.find_all("a")[0].get("href")
                    title = the_one_job.find_all("span", {"class":"sr-only"})[0].text.strip()
                    location = the_one_job.find_all("span", {"class": "job-search-card__location" })[0].text.strip()
                    company = the_one_job.find_all("h4")[0].find("a").text.strip()
                    the_time = the_one_job.find_all("time")[0].text.strip()

                    dict_ = {
                        "title": title,
                        "company": company,
                        "location": location,
                        "time": the_time,
                        "url": url,
                    }

                    if dict_ not in els_jobs:
                        els_jobs.append(dict_)

            time.sleep(5)

            # Verificar si el botón "Ver más empleos" está presente y hacer clic en él
            show_more_button = wd.find_elements(By.CSS_SELECTOR, '.infinite-scroller__show-more-button')
            if show_more_button:
                show_more_button[0].click()
                time.sleep(5)
                
                # Verificar si el contenido de la página ha cambiado después de hacer clic
                current_page_content = wd.page_source

                if current_page_content == previous_page_content:
                    break  # Salir del bucle si no hay más trabajos disponibles

                previous_page_content = current_page_content

            else:
                break  # Salir del bucle si el botón no está presente

        except:
            pass
        time.sleep(5)

    os.system("say -v Monica don escreipin")
    return pd.DataFrame(els_jobs)

df_2 = scraping_cards_on_the_left()
df_2


The version of chrome cannot be detected. Trying with latest driver version


no_of_jobs:  2000


,title,company,location,time,url
0,Data Analyst,HP,Sant Cugat del Vallès,Hace 1 día,https://es.linkedin.com/jobs/view/data-analyst...
1,Data Analyst,Exoticca,Barcelona,Hace 2 meses,https://es.linkedin.com/jobs/view/data-analyst...
2,Senior Data Analyst,TravelPerk,Barcelona,Hace 2 meses,https://es.linkedin.com/jobs/view/senior-data-...
3,Data Analyst,Exoticca,Barcelona,Hace 4 semanas,https://es.linkedin.com/jobs/view/data-analyst...
4,Data Analyst,Danone,Barcelona,Hace 3 meses,https://es.linkedin.com/jobs/view/data-analyst...
5,Data Engineer,Exoticca,Barcelona,Hace 3 semanas,https://es.linkedin.com/jobs/view/data-enginee...
6,Sales Intelligence Senior Analyst,MongoDB,Barcelona,Hace 1 día,https://es.linkedin.com/jobs/view/sales-intell...
7,Data / BI Analyst,AWWG,Barcelona,Hace 17 horas,https://es.linkedin.com/jobs/view/data-bi-anal...
8,Graduate Program Data Engineer - BCN Global IT...,Nestlé IT North America,Esplugues de Llobregat,Hace 2 días,https://es.linkedin.com/jobs/view/graduate-pro...
9,Analista de Datos Junior (Sector Banca / Power...,Butler Scientifics,Barcelona,Hace 2 horas,https://es.linkedin.com/jobs/view/analista-de-...


In [13]:
df_2.to_csv('ultimo_ultimo.csv', index= False)

df_2.to_csv('ultimo_funcion_1.csv', index=False)
df_last = pd.read_csv('ultimo_funcion_1.csv')
df_primerexito = pd.read_csv('primer_exito_bro_to_wapo.csv')
df_medio = pd.read_csv('all_data_analysts_jobs175.csv')
df_primero = pd.read_csv('all_data_analysts_jobs.csv')
df_combined = pd.concat([df_last, df_primero, df_medio, df_primerexito]).drop_duplicates()
df_combined.to_csv('first_function_datajobs.csv', index=False)
df_combined_2 = df_combined.reset_index(drop=True)


In [15]:
df_last = pd.read_csv('ultimo_funcion_1.csv')


In [16]:
df_primero = pd.read_csv('all_data_analysts_jobs.csv')


In [17]:
df_medio = pd.read_csv('all_data_analysts_jobs175.csv')


In [18]:
df_primerexito = pd.read_csv('primer_exito_bro_to_wapo.csv')


In [19]:
df_combined = pd.concat([df_last, df_primero, df_medio, df_primerexito, df_2]).drop_duplicates()


In [24]:
df_final_url = pd.read_csv('csv_urls.csv')

In [21]:
df_combined_2.to_csv('csv_urls_2.csv', index=False)


In [25]:
df_final_url.url[0]

'https://es.linkedin.com/jobs/view/data-analyst-at-exoticca-3622172076?refId=jWf2Fweu6LwaRUAHa5CNVQ%3D%3D&trackingId=vzc7hXRuY8DFO0xPl1w3VA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'

# Scrapear detalles del job

options = webdriver.ChromeOptions()
options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
wd = webdriver.Chrome(options=options)
url_job = "https://es.linkedin.com/jobs/view/data-analyst-at-coniq-3652827423?refId=LmHLqc2XCC2YJGeJ%2BOJlfw%3D%3D&trackingId=CtHhJ%2Fe3ZQTxs8KZpryElw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card"
wd.get(url_job)

#Description’: jd
#Seniority
#Type’: emp_type,
#Function’: job_func,

def logging_in (url):
    
    # 1. Initializing driver
    options = webdriver.ChromeOptions()
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    wd = webdriver.Chrome(options=options)
    wd.get(url)
    
    # 2. Try to log-in
    
    try:
        time.sleep(5)
        driver.find_element(By.XPATH,'/html/body/header/nav/div/a[2]').click()
        mail = 'cocazapata.21@gmail.com'
        pw = 'fcbarcelona21'
        time.sleep(3)
        print("Logging in")

        username = wait.until(EC.visibility_of_element_located((By.ID, 'username'))).send_keys(mail)
        password = wait.until(EC.visibility_of_element_located((By.ID, 'password'))).send_keys(pw)
        password.send_keys(Keys.RETURN)
        time.sleep(2)
        return wd.page_source
        
    # 3. Otherwise: try to just get the content
        
    except:
        print("Not logging in")
        time.sleep(2)
        return wd.page_source

import requests
from bs4 import BeautifulSoup
import time

def info_un_job(url):
    
    response = logging_in (url)
    soup = BeautifulSoup(response, 'html.parser')
    descripcion = soup.find('div', {"class":'description__text description__text--rich'}).get_text(strip=True)
    nivel_antiguedad = soup.find('span', {"class":"description__job-criteria-text description__job-criteria-text--criteria"}).get_text(strip=True)
    sectores = soup.find('span', {"class":"description__job-criteria-text description__job-criteria-text--criteria"})
    
    dict_ = {
        "description": descripcion,
        "nivel_antiguedad": nivel_antiguedad,
        "sectores": sectores
    }
    
    return dict_
    

url = 'https://es.linkedin.com/jobs/view/data-analyst-at-coniq-3652827423?refId=LmHLqc2XCC2YJGeJ%2BOJlfw%3D%3D&trackingId=CtHhJ%2Fe3ZQTxs8KZpryElw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'

resultado = info_un_job(url)
resultado


In [28]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import datetime
import os
import pandas as pd
from selenium import webdriver
import os
import pandas as pd
import time
import random
import getpass
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urlencode
from urllib.parse import quote

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException



def logging_in(url):
    options = webdriver.ChromeOptions()
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    wd = webdriver.Chrome(options=options)
    wd.get(url)

    try:
        time.sleep(3)
        wd.find_element(By.XPATH, '/html/body/header/nav/div/a[2]').click()
        mail = 'cocazapata.21@gmail.com'
        pw = 'fcbarcelona21'
        time.sleep(3)
        print("Logging in")

        username = wd.find_element(By.ID, 'username')
        username.send_keys(mail)
        password = wd.find_element(By.ID, 'password')
        password.send_keys(pw)
        password.send_keys(Keys.RETURN)
        time.sleep(5)
        return wd
    except:
        print("Not logging in")
        time.sleep(4)
        return wd


def info_un_job(url):
    wd = logging_in(url)
    
    response = wd.page_source
    soup = BeautifulSoup(response, 'html.parser')
    
    company = soup.find('span', {"class":"jobs-unified-top-card__company-name"}).get_text(strip=True)
    position = soup.find('h1', {"class":"t-24 t-bold jobs-unified-top-card__job-title"}).get_text(strip=True)
    location = soup.find('span', {"class":"jobs-unified-top-card__bullet"}).get_text(strip=True)
    workplace = soup.find('span', {"class":"jobs-unified-top-card__workplace-type"}).get_text(strip=True)
    type_and_level = soup.find("li", {"class": "jobs-unified-top-card__job-insight"}).find("span").text.strip()
    description = soup.find('div', {"class": 'jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch'}).get_text(strip=True)
    today = datetime.today().strftime('%d-%m-%Y')
    link_text = soup.find('a', class_='app-aware-link').text.strip()
    #workers = soup.find("li", {"class": "jobs-unified-top-card__job-insight"}).find("span").text.strip()
    #num_alumnis = link_text.split()[0]
    applicants = soup.find('span', class_='jobs-unified-top-card__applicant-count')
    applicants = applicants.text.strip() if applicants else None
    
    dict_2 = {
        "company" : company,
        "position": position,
        "location": location,
        "workplace" : workplace,
        "type_and_level": type_and_level,
        "description": description,
        "today": today,
        "url": url,
        "applicants": applicants,
        #"workers": workers,
         #"num_alumnis": num_alumnis,
        
    }
    
    os.system("say -v Monica don escreipin")

    return dict_2

url = 'https://es.linkedin.com/jobs/view/data-analyst-at-exoticca-3622172076?refId=jWf2Fweu6LwaRUAHa5CNVQ%3D%3D&trackingId=vzc7hXRuY8DFO0xPl1w3VA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'
resultado = info_un_job(url)
df_test = pd.DataFrame([resultado])
df_test

df_test

The version of chrome cannot be detected. Trying with latest driver version


Logging in


AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
resultado

In [ ]:
import pandas as pd

job_data = pd.DataFrame({
    'ID': job_id,
    'Date': date,
    'Company': company_name,
    'Title': job_title,
    'Location': location,
    'Description': jd,
    'Level': seniority,
    'Type': emp_type,
    'Function': job_func,
    'Industry': industries,
    'Link': job_link
})

# Limpiar la columna "Description"
job_data['Description'] = job_data['Description'].str.replace('\n', ' ')

job_data.to_excel('LinkedIn Job Data_Data Scientist.xlsx', index=False)


In [ ]:
job_data = pd.DataFrame({‘ID’: job_id,
‘Date’: date,
‘Company’: company_name,
‘Title’: job_title,
‘Location’: location,
'Description’: jd,
‘Level’: seniority,
‘Type’: emp_type,
‘Function’: job_func,
‘Industry’: industries,
‘Link’: job_link
})
# cleaning description column
job_data[‘Description’] = job_data[‘Description’].str.replace(‘\n’,’ ‘)
job_data.to_excel('LinkedIn Job Data_Data Scientist.xlsx', index = False)